# Cross-Sections

In [1]:
import uxarray as ux
import geoviews.feature as gf
import cartopy.crs as ccrs
import geoviews as gv

projection = ccrs.Robinson()

## Data

In [2]:
base_path = "../../test/meshfiles/ugrid/outCSne30/"
grid_path = base_path + "outCSne30.ug"
data_path = base_path + "outCSne30_vortex.nc"

uxds = ux.open_dataset(grid_path, data_path)

uxds

<xarray.UxDataset> Size: 43kB
Dimensions:  (n_face: 5400)
Dimensions without coordinates: n_face
Data variables:
    psi      (n_face) float64 43kB ...

In [3]:
uxds.uxgrid.normalize_cartesian_coordinates()

## Constant Latitude

In [7]:
%%time
lat = 1.0

uxda_constant_lat = uxds["psi"].constant_latitude_cross_section(lat)

CPU times: user 5.89 ms, sys: 2.29 ms, total: 8.18 ms
Wall time: 6.02 ms


In [8]:
constant_lat_line = gv.Path([(-180, lat), (180, lat)]).opts(
    projection=projection, color="grey", line_width=2, alpha=0.5
)

In [9]:
(
    uxda_constant_lat.plot(
        rasterize=False,
        backend="bokeh",
        cmap="inferno",
        projection=projection,
        global_extent=True,
        coastline=True,
    )
    * gf.grid(projection=projection)
    * constant_lat_line
)

:Overlay
   .Polygons.I  :Polygons   [Longitude,Latitude]   (psi)
   .Coastline.I :Feature   [Longitude,Latitude]
   .Grid.I      :Feature   [Longitude,Latitude]
   .Path.I      :Path   [Longitude,Latitude]

## Constant Longitude

## Arbitrary Great Circle Arc (GCA)